In [1]:
# StateMaker.py

class StateMaker:
    def __init__(self, state, n, m):
        # Parsing the input state string
        positions = list(map(int, state.split(',')))
        self.agents = [positions[i:i+2] for i in range(0, 2*n, 2)]
        self.defenses = [positions[i:i+2] for i in range(2*n, 2*n+2*m, 2)]
        self.n = n
        self.m = m
        self.gridsize = 5

    def get_possible_moves(self, agent):
        # Define possible moves
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right
        possible_moves = []
        for dx, dy in directions:
            new_x, new_y = agent[0] + dx, agent[1] + dy
            # Check if move is within the grid
            if 0 <= new_x < self.gridsize and 0 <= new_y < self.gridsize:
                possible_moves.append((new_x, new_y))
        return possible_moves

    def manhattan_distance(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    def generate_next_states(self):
        next_states = []
        all_moves = [self.get_possible_moves(agent) for agent in self.agents]

        def recurse(index, current_state):
            if index == len(all_moves):
                # Check for any attacks on defenses
                defenses_pos = [d[:] for d in self.defenses]  # Create a copy of the defenses positions
                for i, agent_pos in enumerate(current_state):
                    for j, defense_pos in enumerate(defenses_pos):
                        if agent_pos == tuple(defense_pos):
                            defenses_pos[j] = [-1, -1]

                state_str = ', '.join([f"{pos[0]}, {pos[1]}" for pos in (current_state + defenses_pos)])
                next_states.append(state_str)
                return

            for move in all_moves[index]:
                if move not in current_state:  # Ensure no overlap among agents
                    recurse(index + 1, current_state + [move])

        recurse(0, [])
        return next_states


In [2]:
initial_state = "1, 1, 4, 4, 1, 0, 3, 3"
state_maker = StateMaker(initial_state, 2, 2)
next_states = state_maker.generate_next_states()
for state in next_states:
    print(state)


0, 1, 3, 4, 1, 0, 3, 3
0, 1, 4, 3, 1, 0, 3, 3
2, 1, 3, 4, 1, 0, 3, 3
2, 1, 4, 3, 1, 0, 3, 3
1, 0, 3, 4, -1, -1, 3, 3
1, 0, 4, 3, -1, -1, 3, 3
1, 2, 3, 4, 1, 0, 3, 3
1, 2, 4, 3, 1, 0, 3, 3


In [3]:
class StateMaker:
    def __init__(self, state, n, m, p):
        # Initialize StateMaker with the current state, number of agents (n), number of defenses (m),
        # and the probability of a defense hitting an agent (p).
        # The state is parsed to extract positions of agents and defenses.
        positions = list(map(int, state.split(',')))
        self.agents = [positions[i:i+2] for i in range(0, 2*n, 2)]  # Agent positions
        self.defenses = [positions[i:i+2] for i in range(2*n, 2*n+2*m, 2)]  # Defense positions
        self.n = n  # Number of agents
        self.m = m  # Number of defenses
        self.p = p  # Probability of being hit

    def get_possible_moves(self, agent):
        # Determine possible moves for an agent, including staying in place.
        # Moves are restricted within the 5x5 grid boundaries.
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (0, 0)]  # Possible directions, including staying still
        possible_moves = []
        for dx, dy in directions:
            new_x, new_y = agent[0] + dx, agent[1] + dy
            # Include the move if it's within the grid or it's the special "hit" case (-1, -1).
            if (0 <= new_x < 5 and 0 <= new_y < 5) or (new_x, new_y) == (-1, -1):
                possible_moves.append((new_x, new_y))
        return possible_moves

    def manhattan_distance(self, point1, point2):
        # Calculate the Manhattan distance between two points.
        return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])

    def calculate_hit_probability(self, agent_pos, defense_positions):
        # Determine the probability of an agent being hit by a defense.
        # If any defense is within a Manhattan distance of 2 or less, the agent has a probability 'p' of being hit.
        for defense_pos in defense_positions:
            if self.manhattan_distance(agent_pos, defense_pos) <= 2:
                return self.p  # Agent is at risk of being hit
        return 0  # Agent is not at risk, so the probability of surviving is 1

    def generate_next_states(self):
        # Generate all possible next states considering the movements of agents and the defense's ability to hit.
        next_states = []
        all_moves = [self.get_possible_moves(agent) for agent in self.agents]  # Possible moves for all agents

        def recurse(index, current_state, probability):
            # Recursive function to explore all combinations of agent moves and calculate the state probabilities.
            if index == len(all_moves):
                # Once all agent moves are processed, calculate the final state string with the cumulative probability.
                defenses_pos = [d[:] for d in self.defenses]  # Copy of defense positions to check for "attacks"
                
                # Calculate cumulative probability considering the defense's ability to hit agents
                cumulative_probability = probability
                for agent_pos in current_state:
                    cumulative_probability *= self.calculate_hit_probability(agent_pos, self.defenses)
                    # Mark defense as "attacked" if agent's final position matches a defense's position
                    for defense_pos in defenses_pos:
                        if agent_pos == tuple(defense_pos):
                            defenses_pos[defenses_pos.index(defense_pos)] = [-1, -1]

                # Construct the state string including the probability and positions of agents/defenses
                state_str = f"{cumulative_probability}: " + ', '.join([f"{pos[0]}, {pos[1]}" for pos in (current_state + defenses_pos)])
                next_states.append(state_str)
                return

            for move in all_moves[index]:
                # Iterate through possible moves, ensuring no agent overlap, but allow "being hit" state
                if move not in current_state or move == (-1, -1):
                    recurse(index + 1, current_state + [move], probability * (1 - self.p if move != (-1, -1) else 1))

        recurse(0, [], 1)  # Start the recursive exploration with an initial probability of 1
        return next_states  # Return the list of all possible next states with their associated probabilities


In [4]:
initial_state = "1, 1, 4, 4, 1, 0, 3, 3"
state_maker = StateMaker(initial_state, 2, 2, 0.2)
next_states = state_maker.generate_next_states()
for state in next_states:
    print(state)


0.025600000000000008: 0, 1, 3, 4, 1, 0, 3, 3
0.025600000000000008: 0, 1, 4, 3, 1, 0, 3, 3
0.025600000000000008: 0, 1, 4, 4, 1, 0, 3, 3
0.025600000000000008: 2, 1, 3, 4, 1, 0, 3, 3
0.025600000000000008: 2, 1, 4, 3, 1, 0, 3, 3
0.025600000000000008: 2, 1, 4, 4, 1, 0, 3, 3
0.025600000000000008: 1, 0, 3, 4, -1, -1, 3, 3
0.025600000000000008: 1, 0, 4, 3, -1, -1, 3, 3
0.025600000000000008: 1, 0, 4, 4, -1, -1, 3, 3
0.025600000000000008: 1, 2, 3, 4, 1, 0, 3, 3
0.025600000000000008: 1, 2, 4, 3, 1, 0, 3, 3
0.025600000000000008: 1, 2, 4, 4, 1, 0, 3, 3
0.025600000000000008: 1, 1, 3, 4, 1, 0, 3, 3
0.025600000000000008: 1, 1, 4, 3, 1, 0, 3, 3
0.025600000000000008: 1, 1, 4, 4, 1, 0, 3, 3


In [5]:
len(next_states)

15